In [0]:
import requests
import base64
import json
import pandas as pd
import pyspark.sql.types as T
import pandas as pd

In [0]:
dbutils.widgets.text("playlist_id", "")
dbutils.widgets.text("table_name_bronze", "")


table_name_bronze = dbutils.widgets.get("table_name_bronze")

In [0]:
def get_access_token():
        client_id = dbutils.secrets.get(scope="spotify_secrets",key="user")
        client_secret = dbutils.secrets.get(scope="spotify_secrets",key="client_secret")    

        # 1. Codificare Base64 pentru Client ID și Client Secret
        auth_string = f"{client_id}:{client_secret}"
        auth_bytes = auth_string.encode("utf-8")
        auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
        auth_url = 'https://accounts.spotify.com/api/token'

        headers = {
            "Authorization": f"Basic {auth_base64}",
            "Content-Type": "application/x-www-form-urlencoded"
        }

        data = {"grant_type": "client_credentials"}
        # 2. API request
        response = requests.post(auth_url, headers=headers, data=data)
        
        if response.status_code == 200:
            return response.json()['access_token']
        else:
            raise Exception("Nu s-a putut reîmprospăta token-ul.")
    

def get_authorization_header(self,token):
    return {"Authorization": f"Basic {token}"}

In [0]:
# --- Datele tale (presupunem că le ai deja) ---
token = get_access_token()
playlist_id = dbutils.widgets.get("playlist_id")
url =  f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"

headers = {"Authorization": f"Bearer {token}",
           "Content-Type": "application/json"
           }

print(f"Încep extragerea melodiilor din playlistul {playlist_id}...")
try:
    # 1. Fă request-ul la API
    response = requests.get(url, headers=headers)

    # 2. Verifică dacă request-ul a avut succes (ex: cod 200 OK)
    response.raise_for_status()

    # 3. Parsează JSON-ul direct într-un dicționar Python
    # ACEASTA ESTE LINIA CHEIE. Nu e nevoie de Spark.
    data = response.json()

    print("Displai response request >>>>>>>>>> \n")

    # 4. Extrage lista de melodii ('items') din datele primite
    playlist_items = data.get("items", [])
    playlist_track = playlist_items[0]["track"]
    print(f"Am găsit {len(playlist_items)} melodii.")

    # 5. (Opțional) Procesează și afișează datele relevante
    tracks_info = []
    for item in playlist_items:
        track = item.get('track')
        if track:
            id = track.get('id')
            track_name = track.get('name')
            popolarity = track.get('popularity')
            duration_ms = track.get('duration_ms')
            track_number = track.get('track_number')
            disc_number = track.get('disc_number')
            
            artist_names = [artist['name'] for artist in track.get('artists', [])]
            artist_id = [artist['id'] for artist in track.get('artists', [])]
            

            tracks_info.append({
                "Plailist_ID" : playlist_id,
                "Id_Track": id,
                "Nume_Melodie": track_name,
                "Artist": ", ".join(artist_names),
                "Id_Artist": ", ".join(artist_id),
                "Popularity": popolarity,
                "Duration_ms": duration_ms,
                "Track_Number": track_number,
                "Disc_Number": disc_number,
                "Album": track.get('album').get('name'),
                "Number_OF_Artist" : len(artist_names)
            })

except requests.exceptions.HTTPError as http_err:
        print(f"Eroare HTTP: {http_err}")
        print(f"Răspuns API: {response.text}")
except Exception as err:
    print(f"A apărut o altă eroare: {err}")    

bronze_playlist_df = spark.createDataFrame(tracks_info)
display(bronze_playlist_df)


       

In [0]:
bronze_playlist_df = spark.createDataFrame(tracks_info)

In [0]:
bronze_playlist_df.write.format("delta").mode("overwrite").option("meregeSchema","true").option("overwriteSchema", "true").saveAsTable(f"spotify_etl.bronze.{table_name_bronze}_{playlist_id}")